In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

In [3]:
from avalanche.benchmarks.datasets.external_datasets.mnist import get_mnist_dataset

mnist_train, mnist_test = get_mnist_dataset(None)

In [14]:
# mnist_train.data.shape
import torch
from torchvision.transforms import Normalize
from torchvision.models import resnet50, resnet18
from torchvision.models import ResNet50_Weights, ResNet18_Weights
# from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

# from sklearn.metrics import accuracy_score
# from statistics import mean

class FeatureExtractor():

    def __init__(self, sigmoid_scaling = 3):
        self.sigmoid_scaling = sigmoid_scaling

        # Create the extraction network
        # self.weights = ResNet50_Weights.DEFAULT
        # rn = resnet50()
        self.weights = ResNet18_Weights.DEFAULT
        rn = resnet18(weights=self.weights)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Use torch FX and load to the device
        self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
        self.mod = self.mod.to(self.device)
        # Evaluation model
        self.mod.eval()
        # Preprocess stage
        self.preprocess = self.weights.transforms()
        self.min = 0.0
        self.max = 32.0
        self.mult_factor = 1 / (self.max - self.min) * 2

    @torch.no_grad()
    def ext_features(self, img):
        # with torch.no_grad():
        img = img.to('cuda')
        prep = self.preprocess(img)
        features = self.mod(prep)['layer4']
        return features

    @torch.no_grad()
    def avg_features(self, features):
        # avg_features = features.mean(dim=1).flatten(start_dim=1).detach().cpu().numpy()
        features = features.detach().mean(dim=1).flatten(start_dim=1)
        # avg_features = (avg_features - self.min) / (self.max - self.min) * 2 - 1
        # avg_features = ((avg_features - self.min) * self.mult_factor - 1) * 3
        return features
    
    @torch.no_grad()
    def norm_features(self, features):
        mean, std = features.mean((0,)), features.std((0,))
        norm = Normalize(mean, std)
        features = norm(features)
        return features
        # avg_features = avg_features.sigmoid().cpu().numpy().transpose()
    
    @torch.no_grad()
    def sigmoid_squash(self, features):
        return ((features - self.min) * self.mult_factor - 1) * self.sigmoid_scaling

    @torch.no_grad()
    def feature_chain(self, img):
        features = ext_features(img)
        features = avg_features(features)
        features = norm_features(features)
        features = sigmoid_squash(features)

In [15]:
fe = FeatureExtractor()

In [ ]:
def SplitMNISTPreprocessed(
    n_experiences: int,
    *,
    return_task_id=False,
    seed: Optional[int] = None,
    fixed_class_order: Optional[Sequence[int]] = None,
    shuffle: bool = True,
    class_ids_from_zero_in_each_exp: bool = False,
    train_transform: Optional[Any] = _default_mnist_train_transform,
    eval_transform: Optional[Any] = _default_mnist_eval_transform,
    dataset_root: Union[str, Path] = None
):
    """
    Creates a CL benchmark using the MNIST dataset.

    If the dataset is not present in the computer, this method will
    automatically download and store it.

    The returned benchmark will return experiences containing all patterns of a
    subset of classes, which means that each class is only seen "once".
    This is one of the most common scenarios in the Continual Learning
    literature. Common names used in literature to describe this kind of
    scenario are "Class Incremental", "New Classes", etc. By default,
    an equal amount of classes will be assigned to each experience.

    This generator doesn't force a choice on the availability of task labels,
    a choice that is left to the user (see the `return_task_id` parameter for
    more info on task labels).

    The benchmark instance returned by this method will have two fields,
    `train_stream` and `test_stream`, which can be iterated to obtain
    training and test :class:`Experience`. Each Experience contains the
    `dataset` and the associated task label.

    The benchmark API is quite simple and is uniform across all benchmark
    generators. It is recommended to check the tutorial of the "benchmark" API,
    which contains usage examples ranging from "basic" to "advanced".

    :param n_experiences: The number of incremental experiences in the current
        benchmark.
        The value of this parameter should be a divisor of 10.
    :param return_task_id: if True, a progressive task id is returned for every
        experience. If False, all experiences will have a task ID of 0.
    :param seed: A valid int used to initialize the random number generator.
        Can be None.
    :param fixed_class_order: A list of class IDs used to define the class
        order. If None, value of ``seed`` will be used to define the class
        order. If non-None, ``seed`` parameter will be ignored.
        Defaults to None.
    :param shuffle: If true, the class order in the incremental experiences is
        randomly shuffled. Default to True.
    :param class_ids_from_zero_in_each_exp: If True, original class IDs
        will be mapped to range [0, n_classes_in_exp) for each experience.
        Defaults to False. Mutually exclusive with the
        ``class_ids_from_zero_from_first_exp`` parameter.
    :param train_transform: The transformation to apply to the training data,
        e.g. a random crop, a normalization or a concatenation of different
        transformations (see torchvision.transform documentation for a
        comprehensive list of possible transformations).
        If no transformation is passed, the default train transformation
        will be used.
    :param eval_transform: The transformation to apply to the test data,
        e.g. a random crop, a normalization or a concatenation of different
        transformations (see torchvision.transform documentation for a
        comprehensive list of possible transformations).
        If no transformation is passed, the default test transformation
        will be used.
    :param dataset_root: The root path of the dataset. Defaults to None, which
        means that the default location for 'mnist' will be used.

    :returns: A properly initialized :class:`NCScenario` instance.
    """

    mnist_train, mnist_test = get_mnist_dataset(dataset_root)

    return nc_benchmark(
        train_dataset=mnist_train,
        test_dataset=mnist_test,
        n_experiences=n_experiences,
        task_labels=return_task_id,
        seed=seed,
        fixed_class_order=fixed_class_order,
        shuffle=shuffle,
        class_ids_from_zero_in_each_exp=class_ids_from_zero_in_each_exp,
        train_transform=train_transform,
        eval_transform=eval_transform,
    )

In [2]:
from torch.utils.data import DataLoader
from julia import Main as jl
from sklearn.metrics import accuracy_score
import ipdb
import torch
# from torchvision.transforms import Lambda
from tqdm import tqdm

from torchvision.models import resnet50, resnet18
from torchvision.models import ResNet50_Weights, ResNet18_Weights
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

from sklearn.metrics import accuracy_score
from statistics import mean

class DDVFAStrategy():
    """DDVFA Strategy"""

    def __init__(self, preprocessed=False):
        jl.project_dir = projectdir()
        jl.eval("using Pkg; Pkg.activate(project_dir)")
        jl.eval("using AdaptiveResonance")
        jl.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        # jl.eval("art = DDVFA(rho_lb=0.5, rho_ub=0.75)")
        jl.eval("art.config = DataConfig(0, 1.0, 49)")
        
        self.preprocessed = preprocessed
        if not self.preprocessed:
            # rn = resnet50()
            self.weights = ResNet18_Weights.DEFAULT
            # rn = resnet18(pretrained=True)
            rn = resnet18(weights=self.weights)
            self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
            self.mod = self.mod.to('cuda')
            self.mod.eval()
            # self.weights = ResNet50_Weights.DEFAULT
            self.preprocess = self.weights.transforms()
            self.min = 0.0
            self.max = 32.0
            self.mult_factor = 1 / (self.max - self.min) * 2

    def ext_features(self, img):
        with torch.no_grad():
            img = img.to('cuda')
            prep = self.preprocess(img)
            features = self.mod(prep)['layer4']
            # avg_features = features.mean(dim=1).flatten(start_dim=1).detach().cpu().numpy()
            avg_features = features.detach().mean(dim=1).flatten(start_dim=1)
            # avg_features = (avg_features - self.min) / (self.max - self.min) * 2 - 1
            avg_features = ((avg_features - self.min) * self.mult_factor - 1) * 3
            avg_features = avg_features.sigmoid().cpu().numpy().transpose()
            # avg_features = features.mean(dim=1).flatten(start_dim=1).cpu().numpy()
            # ipdb.set_trace()
            # avg_features.flatten().detach().numpy()

        return avg_features

    def train(self, experience):
        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            dataset=train_dataset,
            pin_memory=True,
            # batch_size=90,
            batch_size=256,
        )
        print(experience.dataset.__len__())
        for mb in tqdm(train_data_loader):
            data, labels, tasks = mb
            jl.features = self.ext_features(data)
            jl.labels = labels.numpy()
            # ipdb.set_trace()
            jl.eval("train!(art, features, y=labels)")

    def eval(self, experience):
        eval_dataset = experience.dataset
        t = experience.task_label

        eval_data_loader = DataLoader(
            dataset=eval_dataset,
            pin_memory=True,
            # batch_size=90,
            batch_size=256,
        )

        print(experience.dataset.__len__())
        perfs = []
        for mb in tqdm(eval_data_loader):
            data, labels, tasks = mb
            jl.features = self.ext_features(data)
            jl.eval("y_hats = classify(art, features, get_bmu=true)")
            y_hats = jl.y_hats
            # ipdb.set_trace()
            perfs.append(accuracy_score(labels, y_hats))
            # correct += torch.sum(y_hats == labels)
            # total += len(data)
        
        return mean(perfs)
        # j.samples = mb
        # y_hats = j.eval("classify(art, samples)")
        # accuracy_score(y_test, y_hat)

print_allocated_memory()

0.00 GB


In [3]:
from avalanche.benchmarks import SplitMNIST
from avalanche.benchmarks.classic import SplitCIFAR10
from torchvision.transforms import Lambda

trans = Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)
# Benchmark creation
benchmark = SplitMNIST(
    n_experiences=5,
    # n_experiences=2,
    # n_experiences=10,
    train_transform=trans,
    eval_transform=trans,
)
# benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)

# Create the Strategy Instance
cl_strategy = DDVFAStrategy()

# Training Loop
print('Starting experiment...')

results = []
for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results.append(cl_strategy.eval(benchmark.test_stream[exp_id]))
    # results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience  0
11791


 89%|████████▉ | 42/47 [01:12<00:08,  1.73s/it]


KeyboardInterrupt: 

In [5]:
benchmark.test_stream[1].classes_in_this_experience
# benchmark.train_stream[1].classes_in_this_experience

[4, 7]

In [4]:
results

[0.9266000859106529, 0.0]

In [14]:
print(jl.eval("AdaptiveResonance.get_n_weights(art)"))
print(jl.eval("size(art.F2)"))

12862
(12862,)


In [ ]:
a = [1,1,2,3]
b = [1,2,2,3]
from sklearn.metrics import accuracy_score
from statistics import mean

# mean(a)
# accuracy_score(a, b)

In [ ]:
jl.eval("a = 1234")
print(jl.a)

1234
